<a href="https://colab.research.google.com/github/Yogeshkumar170201/Yogeshkumar170201/blob/master/PoseDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ykdahiya","key":"d38b7d9f508bcc40efcc891626f72ca3"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                      title                                     size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------  ---------------------------------------  -----  -------------------  -------------  ---------  ---------------  
themrityunjaypathak/covid-cases-and-deaths-worldwide     Covid Cases and Deaths WorldWide           8KB  2023-02-01 12:22:51          11192        371  1.0              
naveenkumar20bps1137/walmart-sales-analysis              WALMART SALES ANALYSIS                   119KB  2023-03-03 18:02:11           1634         47  0.9411765        
datascientistanna/customers-dataset                      Shop Customer Data                        23KB  2023-02-07 18:42:21           9646        219  1.0              
ulrikthygepedersen/co2-emissions-by-country              CO2 Emissions                            114KB  2023-02-28 12:39:49            839         28

In [ ]:
!kaggle datasets download -d hasyimabdillah/workoutfitness-video

100% 4.32G/4.32G [00:45<00:00, 106MB/s] 
100% 4.32G/4.32G [00:45<00:00, 101MB/s]


In [ ]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:
!unzip /content/workoutfitness-video.zip -d /content/datset

Archive:  /content/workoutfitness-video.zip
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_1.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_10.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_11.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_12.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_13.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_14.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_15.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_16.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_17.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_18.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_19.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_2.mp4  
  inflating: /content/

In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 47.5 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import mediapipe as mp
from google.colab.patches import cv2_imshow
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
# # Load MediaPipe Pose Detection
# mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic

In [ ]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 128, 128

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 20

In [ ]:
def landmarks_extraction(path):
  # # Load video
  # cap = cv2.VideoCapture(path)

  # # Define MediaPipe Pose Detection model
  # pose_detection = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

  # # Loop through video frames
  # while cap.isOpened():
  #     # Read frame
  #     ret, frame = cap.read()
  #     if not ret:
  #         break
      
  #     # Convert frame to RGB
  #     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      
  #     # Detect poses in frame
  #     results = pose_detection.process(frame)
      
  #     # Draw keypoints on frame
  #     if results.pose_landmarks:
  #         mp_drawing = mp.solutions.drawing_utils
  #         mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
      
  #     # Display frame
  #     # cv2_imshow(frame)
      
  #     # Press 'q' to quit
  #     if cv2.waitKey(1) == ord('q'):
  #         break

  # # Release video and destroy windows
  # cap.release()
  # cv2.destroyAllWindows()
  # return results.pose_landmarks
  # Declare a list to store video frames.
  frames_list = []
  # Define MediaPipe Pose Detection model
  # pose_detection = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
  # Read the Video File using the VideoCapture object.
  video_reader = cv2.VideoCapture(path)

  # Get the total number of frames in the video.
  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

  # Calculate the the interval after which frames will be added to the list.
  skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

  # Iterate through the Video Frames.
  
  for frame_counter in range(SEQUENCE_LENGTH):

      # Set the current frame position of the video.
      video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

      # Reading the frame from the video. 
      success, frame = video_reader.read() 

      # Check if Video frame is not successfully read then break the loop
      if not success:
          break

      # Resize the Frame to fixed height and width.
      # resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
      
      # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
      # normalized_frame = resized_frame / 255
      
      # Append the normalized frame into the frames list
      frames_list.append(frame)

  # Release the VideoCapture object. 
  video_reader.release()
  marks = []
  with mp_holistic.Holistic() as holistic:
    for i in frames_list:
      # Convert frame to RGB
      frame = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)

      # Detect poses in frame
      results = holistic.process(frame)
      pose_landmarks = results.pose_landmarks
      # print(pose_landmarks)
      if pose_landmarks is not None:
        landmarks = [(lmk.x, lmk.y, lmk.z) for lmk in pose_landmarks.landmark]
        marks.append(np.array(landmarks))
  # Return the frames list.
  marks = np.array(marks)
  return marks

In [ ]:
# Specify the directory containing the dataset. 
DATASET_DIR = "datset"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = os.listdir('/content/datset')
CLASSES_LIST = CLASSES_LIST[:5]

In [ ]:

def create_dataset():
  marks = []
  labels = []
  for class_index, class_name in enumerate(CLASSES_LIST):
    print(class_index, class_name)
    # Display the name of the class whose data is being extracted.
    print(f'Extracting Data of Class: {class_name}')

    # Get the list of video files present in the specific class name directory.
    files_list = os.listdir(os.path.join(DATASET_DIR, class_name))

    # Iterate through all the files present in the files list.
    for file_name in files_list:

      # Get the complete video path.
      video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

      # Extract the landmarks of the video file.
      landmarks = landmarks_extraction(video_file_path)
      marks.append(np.array(landmarks))
      labels.append(class_index)
  return marks, labels


In [ ]:
marks, labels = create_dataset()

0 bench press
Extracting Data of Class: bench press
1 incline bench press
Extracting Data of Class: incline bench press
2 push-up
Extracting Data of Class: push-up
3 hammer curl
Extracting Data of Class: hammer curl
4 chest fly machine
Extracting Data of Class: chest fly machine


In [ ]:
labels = np.array(labels)
marks = np.array(marks)
x = marks
y = labels

<ipython-input-18-021d6fe70071>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  marks = np.array(marks)


In [ ]:
for i in range(0, len(x)):
  if x[i].shape!=(20,99) and x[i].shape!=(20,33,3):
    x = np.delete(x, i)
    y = np.delete(y, i)
    print(x[i].shape)

In [ ]:
for i in range(0, len(x)):
  if x[i].shape!=(20,99):
    x[i] = x[i].reshape(x[i].shape[0], -1)

In [ ]:
print(x.shape)

(141,)


In [ ]:
print(y.shape)

(141,)


In [ ]:
for i in range(len(x)):
  print(x[i].shape)

(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(20, 99)
(

In [ ]:
z = y
print(y.shape)

(141,)


In [ ]:
z = tf.keras.utils.to_categorical(z).astype(int)
print(z.shape)

(141, 5)


In [ ]:
print(z)

[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]

In [ ]:
# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(20, 99)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                41984     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 44,229
Trainable params: 44,229
Non-trainable params: 0
_________________________________________________________________


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, z, test_size=0.2, random_state=42)

In [ ]:
for i in range(len(X_train)):
  X_train[i] = tf.constant(X_train[i])
for i in range(len(X_test)):
  X_test[i] = tf.constant(X_test[i])

In [ ]:
x_train = tf.TensorArray(dtype=tf.float64, size=0, dynamic_size=True)
x_test = tf.TensorArray(dtype=tf.float64, size=0, dynamic_size=True)

for i in range(len(X_train)):
  x_train = x_train.write(i, tf.constant(X_train[i]))

for i in range(len(X_test)):
  x_test = x_test.write(i, tf.constant(X_test[i]))
y_train = tf.constant(y_train)
y_test = tf.constant(y_test)
print(type(x_train))
print(type(x_test))

<class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>
<class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>


In [ ]:
# x_train = x_train.stack()
# x_test = x_test.stack()
# y_train = y_train.stack()
# y_test = y_test.stack()

AttributeError: ignored

In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=100)

Epoch 1/50
2/2 [==============================] - 0s 12ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 2/50
2/2 [==============================] - 0s 13ms/step - loss: 0.0078 - accuracy: 1.0000
Epoch 3/50
2/2 [==============================] - 0s 11ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 4/50
2/2 [==============================] - 0s 13ms/step - loss: 0.0112 - accuracy: 1.0000
Epoch 5/50
2/2 [==============================] - 0s 13ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 6/50
2/2 [==============================] - 0s 23ms/step - loss: 0.0200 - accuracy: 1.0000
Epoch 7/50
2/2 [==============================] - 0s 11ms/step - loss: 0.0100 - accuracy: 1.0000
Epoch 8/50
2/2 [==============================] - 0s 12ms/step - loss: 0.0134 - accuracy: 1.0000
Epoch 9/50
2/2 [==============================] - 0s 13ms/step - loss: 0.0112 - accuracy: 1.0000
Epoch 10/50
2/2 [==============================] - 0s 11ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 11/50
2/2 [============

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 27ms/step - loss: 0.8047 - accuracy: 0.8621


In [ ]:
print(loss)
print(accuracy)

0.8047085404396057
0.8620689511299133
